In [1]:
import tweepy
import json
import os
import re

from IPython.display import display, clear_output

In [2]:
twitter_credentials = json.load(open('keys.json', 'r'))['twitter1']
CONSUMER_KEY = twitter_credentials['consumer_key']
CONSUMER_SECRET = twitter_credentials['consumer_secret']
token_key    = twitter_credentials['token_key']
token_secret = twitter_credentials['token_secret']

In [3]:
# Authenticate twitter Api
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
# auth.set_access_token(token_key, token_secret)

In [13]:
dir_path = './tweets_NY_150km'
all_tweets = []
for file_name in os.listdir(dir_path):
    file_path = dir_path + '/' + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        all_tweets.append(json.load(file)[0])
    
ids = [tweet['id'] for tweet in all_tweets]
last_id = max(ids)
first_id = min(ids)
print('First :', first_id)
print('Last  :', last_id)

First : 1172558832842792960
Last  : 1174739452285075457


In [14]:
tweet_per_file = 1000
max_n_files = 100
i = 0
i_file = 4


c = tweepy.Cursor(api.search, 
                  q='*', 
                  geocode='40.7128,74.0060,150km', 
                  tweet_mode='extended', 
                  lang='en', 
                  since_id=last_id,
#                  max_id=first_id,
                  result_type = 'recent'
                 )

tweets = []
for tweet in c.items():
    
    #get full text for retweets and normal tweets too
    try:
        text = tweet.retweeted_status.full_text
    except AttributeError:
        text = tweet.full_text
        
    #save certain attributes (other than text)
    tweets.append(
      {
          'id':tweet.id,
          'text':text,
          'created_at':str(tweet.created_at),
          'author_name':tweet.author.name,
      })
    
    #save every #tweet_per_file number of tweets to a json
    i += 1
    if i > (tweet_per_file-1):
        with open(dir_path + '/' + 'tweets_{:03d}.json'.format(i_file), 'w', encoding='utf-8') as file:
            json.dump(tweets, file, ensure_ascii=False, indent=4)
        i_file += 1
        i = 0
        tweets = []
    
    clear_output(wait=True)
    display('{}/{}'.format(i_file, i))

    if i_file > (max_n_files):
        break
        